In [1]:
import random
import sys
import numpy as np
import copy
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
class Move(object):
  def __init__(self, color, x1, y1, x2, y2):
    self.color = color
    self.x1 = x1
    self.y1 = y1
    self.x2 = x2
    self.y2 = y2  

  def isValidMove(self,board,mute=False):
    if not board.isOnBoard(self.x1,self.y1) or not board.isOnBoard(self.x2,self.y2):
      if not mute:
        print('Not on board')
      return False
    if board.board [self.x2] [self.y2] == 0 or board.board [self.x1] [self.y1] == 0:
      if not mute:
        print('This is an empty cell')
      return False
    if self.color != board.board [self.x1] [self.y1]:
      if not mute:
        print('Choose a tile from your color')
      return False
    if self.color == white:
      if board.board [self.x2] [self.y2] != black:
        if not mute:
          print('You can only eat opposite color')
        return False
    if self.color == black:
      if board.board [self.x2] [self.y2] != white:
        if not mute:
          print('You can only eat opposite color')
        return False
    if abs(self.x2 - self.x1) + abs(self.y2 - self.y1) != 1 :
      if not mute:
        print("You can only eat adjacent tile")
      return False
    return True

In [11]:
Dx = 5
Dy = 5
white = 1
black = 2
colors = ['grey', 'white', 'black']
bounds = [0,1,2,3]

cmap = mpl.colors.ListedColormap(colors)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

class Board(object):

  def __init__(self,turn):
    self.h = 0
    self.turn = turn
    # Creates a brand new, blank board data structure.
    self.board = np.zeros ((Dx, Dy))
    
  def drawBoard(self):
    print(self.board)
    #plt.imshow(self.board, interpolation='none', cmap=cmap, norm=norm)

  def resetBoard(self):
    # Blanks out the board it is passed, except for the original starting position.
    self.board = np.zeros ((Dx, Dy))
    for x in range(Dx):
      for y in range(Dy):
        if (x+y)%2 == 0 :
          self.board[x][y] = white
        else:
          self.board[x][y] = black

  def isOnBoard(self,x, y):
    # Returns True if the coordinates are located on the board.
    return x in np.arange(Dx) and y in np.arange(Dy)

  def getValidMoves(self):
    # Returns a list of [x,y] lists of valid moves for the given player on the given board.
    validMoves = []
    for x in range(Dx):
      for y in range(Dy):
        if self.board[x][y] == self.turn:
          for k in [-1, 0 ,1]:
            for l in [-1, 0, 1]:
              m = Move (self.turn, x, y, x + k, y + l)
              if m.isValidMove(self,mute=True) != False:
                validMoves.append(m)
    return validMoves

  def getScoreOfBoard(self):
    # Determine the score by counting the tiles. Returns a dictionary with keys 'X' and 'O'.
    whites = 0.5
    blacks = 0.5
    l = self.getValidMoves()
    if len(l) == 0 :
      if self.turn == black:
        whites = 1
        blacks = 0
      else:
        blacks = 1
        whites = 0
    return {white : whites, black : blacks}
  

  def terminal (self):
    return self.getScoreOfBoard()[white] != 0.5


  def makeMove(self, move):
    # Place the tile on the board at xstart, ystart, and flip any of the opponent's pieces.
    # Returns False if this is an invalid move, True if it is valid.
    col = int (self.board [move.x2] [move.y2])
    if col != 0:
      self.h = self.h ^ hashTable [col] [move.x2] [move.y2]
    self.h = self.h ^ hashTable [move.color] [move.x2] [move.y2]
    self.h = self.h ^ hashTable [move.color] [move.x1] [move.y1]
    self.h = self.h ^ hashTurn
    self.board [move.x1] [move.y1] = 0
    self.board [move.x2] [move.y2] = move.color
    if (self.turn == white):
      self.turn = black
    else:
      self.turn = white
  
  def makeRandomMove(self):
    moves = self.getValidMoves()
    n = random.randint (0, len (moves) - 1)
    self.makeMove(moves[n])


  def getBoardCopy(self):
    # Make a duplicate of the board list and return the duplicate.
    return copy.deepcopy(self)

  def getPlayerMove(self):
    # Let the player type in their move.
    # Returns the move as [x, y] (or returns the strings 'hints' or 'quit')
    DIGITS = [str(i+1) for i in range(Dx)]
    while True:
      print('Enter your move, or type quit to end the game.')
      print('The move has to be a sequence of 4 digits x1y1x2y2 for example 1415 takes the pawn at (1,5) with the pawn (1,4)')
      move = input().lower()
      if move == 'quit':
        return 'quit'
      if len(move) == 4 and move[0] in DIGITS and move[1] in DIGITS and move[2] in DIGITS and move[3] in DIGITS:
        x1 = int(move[0]) - 1
        y1 = int(move[1]) - 1
        x2 = int(move[2]) - 1
        y2 = int(move[3]) - 1
        m = Move (self.turn,x1,y1,x2,y2)
        if m.isValidMove(self) == False:
          continue
        else:
          break
      else:
        print('That is not a valid move. Type the x digit (1-8), then the y digit (1-8).')
        print('For example, 81 will be the top-right corner.')
    return m

  def getComputerMove(self,algo):
    # Given a board and the computer's tile, determine where to
    assert algo == 'UCT' or algo =='nested'
    if algo =='UCT':
      bestMove= BestMoveUCT(self,100)
    elif algo == 'nested':
      bestMove= BestMoveUCTNested(self,100)
    return bestMove

  def showPoints(self,playerTile, computerTile):
    # Prints out the current score.
    scores = self.getScoreOfBoard()
    print('You have %s points. The computer has %s points.' % (scores[playerTile], scores[computerTile]))
    print('Welcome to Reversi!')


  def playout (self):
    while (True):
      moves = self.getValidMoves()
      if self.terminal ():
        return self.getScoreOfBoard()
      n = random.randint (0, len (moves) - 1)
      self.makeMove(moves[n])

  def nestedDiscountedPlayout (self, t):
    while (True):
      if self.terminal():
        return self.getScoreOfBoard()[computerTile]/ (t + 1)
      moves = self.getValidMoves()
      bestMove = moves [0]
      best = -2
      for i in range (len (moves)):
        b = copy.deepcopy (self)
        b.makeMove (moves [i])
        s = b.discountedPlayout (t)
        if self.turn == playerTile:
          s = -s
        if s > best:
          best = s
          bestMove = moves [i]
      self.makeMove(bestMove)
      t = t + 1

  def discountedPlayout (self, t):
    while (True):
      moves = self.getValidMoves()
      if self.terminal ():
        return self.getScoreOfBoard()[computerTile]/ (t + 1)
      n = random.randint (0, len (moves) - 1)
      self.makeMove(moves [n])
      t = t + 1

def whoGoesFirst():
  return white

def playAgain():
  # This function returns True if the player wants to play again, otherwise it returns False.
  print('Do you want to play again? (yes or no)')
  return input().lower().startswith('y')

def enterPlayerTile():
  # Lets the player type which tile they want to be.
  # Returns a list with the player's tile as the first item, and the computer's tile as the second.
  tile = 0
  while not (tile == white or tile == black):
    print('Do you want to play the whites or the blacks?')
    tile = 2- int(input().lower().startswith('white'))
  if tile == white:
    print('You play the whites')
    return [white, black]
  else:
    print('You play the blacks')
    return [black, white]

In [4]:
hashTable = []
for k in range (3):
    l = []
    for i in range (Dx):
        l1 = []
        for j in range (Dy):
            l1.append (random.randint (0, 2 ** 64))
        l.append (l1)
    hashTable.append (l)
hashTurn = random.randint (0, 2 ** 64)

MaxLegalMoves = int(0.5*(4*(Dx-2) * (Dy-2) + 3 * 2 * (Dx - 2) + 3 * 2 * (Dy - 2) + 4 * 2))
Table = {}

def add (board):
    nplayouts = [0.0 for x in range (MaxLegalMoves)]
    nwins = [0.0 for x in range (MaxLegalMoves)]
    Table [board.h] = [0, nplayouts, nwins]

def look (board):
    return Table.get(board.h, None)


In [5]:
def UCT(board):
  if board.terminal():
    return board.getScoreOfBoard()
  t = look(board)
  if t:
    bestValue = -1
    best =0
    moves = board.getValidMoves()
    for i in range(0,len(moves)):
      val= 10000000
      if t[1][i]>0:
        Q = t[2][i]/t[1][i]
        if board.turn == playerTile :
          Q = 1 - Q
        val = Q + 0.4* np.sqrt(np.log(t[0])/t[1][i])
      if val > bestValue:
        bestValue = val
        best = i
    #print(moves [best]) 
    board.makeMove(moves [best])
    res = UCT (board)
    t [0] += 1
    t [1] [best] += 1
    t [2] [best] += res[computerTile]
    return res
  else:
    add(board)
    return board.playout()

def BestMoveUCT (board, n):
    global Table
    Table = {}
    for i in range (n):
        b1 = copy.deepcopy (board)
        res = UCT (b1)
    t = look (board)
    moves = board.getValidMoves()
    best = moves [0]
    bestValue = t [1] [0]
    for i in range (1, len(moves)):
        if (t [1] [i] > bestValue):
            bestValue = t [1] [i]
            best = moves [i]
    return best

In [6]:
def UCTNested (board, t1):
    if board.terminal ():
      return board.getScoreOfBoard()[computerTile] / (t1 + 1)
    t = look (board)
    if t != None:
        bestValue = -1000000.0
        best = 0
        moves = board.getValidMoves()
        for i in range (0, len (moves)):
            val = 1000000.0
            if t [1] [i] > 0:
                Q = t [2] [i] / t [1] [i]
                if board.turn == playerTile:
                    Q = 1-Q
                val = Q + 0.4 * np.sqrt (np.log(t [0]) / t [1] [i])
            if val > bestValue:
                bestValue = val
                best = i
        board.makeMove(moves [best])
        res = UCTNested (board, t1 + 1)
        t [0] += 1
        t [1] [best] += 1
        t [2] [best] += res
        return res
    else:
        add (board)
        return board.nestedDiscountedPlayout(t1)

def BestMoveUCTNested (board, n):
    global Table
    Table = {}
    for i in range (n):
        b1 = copy.deepcopy (board)
        res = UCTNested(b1,0)
    t = look (board)
    moves = board.getValidMoves()
    best = moves [0]
    bestValue = t [1] [0]
    for i in range (1, len(moves)):
        if (t [1] [i] > bestValue):
            bestValue = t [1] [i]
            best = moves [i]
    return best

In [ ]:
def playGame(algo):
  while True:
    # Reset the board and game.
    turn = whoGoesFirst()
    mainBoard = Board(turn)
    mainBoard.resetBoard()
    global playerTile, computerTile 
    playerTile, computerTile = white,black
    while True:
      if mainBoard.turn == white:
        if playerTile == white :
          #mainBoard.drawBoard()
          mainBoard.makeRandomMove()
          if mainBoard.getValidMoves() == []:
              break
        else:
          # Computer's turn.
          #mainBoard.drawBoard()
          computer_move = mainBoard.getComputerMove(algo)
          mainBoard.makeMove(computer_move)
          if mainBoard.getValidMoves() == []:
            break
      else:
        if playerTile == black :
          #mainBoard.drawBoard()
          mainBoard.makeRandomMove()
          if mainBoard.getValidMoves() == []:
            break
        else:    
          # Computer's turn.
          #mainBoard.drawBoard()
          computer_move = mainBoard.getComputerMove(algo)
          mainBoard.makeMove(computer_move)
          if mainBoard.getValidMoves() == []:
            break
    # Display the final score.
    mainBoard.drawBoard()
    scores = mainBoard.getScoreOfBoard()
    print('X scored %s points. O scored %s points.' % (scores[white], scores[black]))
    if scores[playerTile] > scores[computerTile]:
      print('You beat the computer by %s points! Congratulations!' % (scores[playerTile] - scores[computerTile]))
      winner = playerTile
    elif scores[playerTile] < scores[computerTile]:
      print('You lost. The computer beat you by %s points.' % (scores[computerTile] - scores[playerTile]))
      winner = computerTile
    break
  return winner

In [ ]:
def playNRandomGames(n,algo):
  wwin = 0
  bwin = 0
  for _ in range(n):
    winner = playGame(algo)
    if winner == white:
      wwin +=1
    else:
      bwin +=1
  print(f"Whites won {wwin} games")
  print(f"Blacks won {bwin} games")


In [ ]:
playNRandomGames(10, 'nested')

[[2. 0. 1. 0. 0.]
 [2. 0. 1. 0. 2.]
 [0. 2. 0. 1. 0.]
 [0. 0. 2. 0. 1.]
 [1. 0. 0. 1. 0.]]
X scored 0 points. O scored 1 points.
You lost. The computer beat you by 1 points.
[[0. 2. 0. 0. 1.]
 [2. 0. 0. 0. 1.]
 [0. 2. 0. 1. 0.]
 [1. 0. 0. 0. 1.]
 [0. 0. 0. 2. 0.]]
X scored 0 points. O scored 1 points.
You lost. The computer beat you by 1 points.
[[2. 0. 0. 0. 1.]
 [0. 2. 2. 0. 1.]
 [2. 0. 0. 1. 0.]
 [0. 0. 2. 0. 1.]
 [0. 1. 0. 1. 0.]]
X scored 0 points. O scored 1 points.
You lost. The computer beat you by 1 points.
[[2. 0. 2. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 2. 0. 0. 0.]
 [2. 0. 1. 0. 2.]
 [0. 1. 1. 1. 0.]]
X scored 0 points. O scored 1 points.
You lost. The computer beat you by 1 points.
[[2. 0. 2. 0. 1.]
 [0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [2. 0. 1. 0. 1.]
 [2. 0. 1. 1. 0.]]
X scored 0 points. O scored 1 points.
You lost. The computer beat you by 1 points.
[[2. 0. 0. 0. 0.]
 [0. 1. 1. 0. 2.]
 [0. 0. 1. 0. 0.]
 [2. 0. 0. 1. 1.]
 [2. 0. 0. 0. 0.]]
X scored 0 points. O scored 1 points.
Y

In [13]:
def playAgainstComputer(algo):
  while True:
    # Reset the board and game.
    turn = whoGoesFirst()
    print('The whites will go first.')
    mainBoard = Board(turn)
    mainBoard.resetBoard()
    global playerTile, computerTile
    playerTile, computerTile = enterPlayerTile()
    while True:
      if mainBoard.turn == white:
        if playerTile == white :
          mainBoard.drawBoard()
          move = mainBoard.getPlayerMove()
          if move == 'quit':
            print('Thanks for playing!')
            sys.exit() # terminate the program
          else:
            mainBoard.makeMove(move)
            if mainBoard.getValidMoves() == []:
              break
        else:
          # Computer's turn.
          mainBoard.drawBoard()
          input('Press Enter to see the computer\'s move.')
          computer_move = mainBoard.getComputerMove(algo)
          mainBoard.makeMove(computer_move)
          if mainBoard.getValidMoves() == []:
            break
      else:
        if playerTile == black :
          mainBoard.drawBoard()
          move = mainBoard.getPlayerMove()
          if move == 'quit':
            print('Thanks for playing!')
            sys.exit() # terminate the program
          else:
            mainBoard.makeMove(move)
            if mainBoard.getValidMoves() == []:
              break
        else:    
          # Computer's turn.
          mainBoard.drawBoard()
          input('Press Enter to see the computer\'s move.')
          computer_move = mainBoard.getComputerMove(algo)
          mainBoard.makeMove(computer_move)
          if mainBoard.getValidMoves() == []:
            break
          else:
            turn = white
    # Display the final score.
    mainBoard.drawBoard()
    scores = mainBoard.getScoreOfBoard()
    print('X scored %s points. O scored %s points.' % (scores[white], scores[black]))
    if scores[playerTile] > scores[computerTile]:
      print('You beat the computer by %s points! Congratulations!' % (scores[playerTile] - scores[computerTile]))
    elif scores[playerTile] < scores[computerTile]:
      print('You lost. The computer beat you by %s points.' % (scores[computerTile] - scores[playerTile]))
    else:
      print('The game was a tie!')
    if not playAgain():
      break

In [14]:
playAgainstComputer('UCT')

The whites will go first.
Do you want to play the whites or the blacks?
white
You play the whites
[[1. 2. 1. 2. 1.]
 [2. 1. 2. 1. 2.]
 [1. 2. 1. 2. 1.]
 [2. 1. 2. 1. 2.]
 [1. 2. 1. 2. 1.]]
Enter your move, or type quit to end the game.
The move has to be a sequence of 4 digits x1y1x2y2 for example 1415 takes the pawn at (1,5) with the pawn (1,4)
1112
[[0. 1. 1. 2. 1.]
 [2. 1. 2. 1. 2.]
 [1. 2. 1. 2. 1.]
 [2. 1. 2. 1. 2.]
 [1. 2. 1. 2. 1.]]
Press Enter to see the computer's move.
[[0. 1. 1. 2. 1.]
 [2. 1. 2. 1. 2.]
 [1. 2. 1. 2. 1.]
 [2. 1. 0. 1. 2.]
 [1. 2. 2. 2. 1.]]
Enter your move, or type quit to end the game.
The move has to be a sequence of 4 digits x1y1x2y2 for example 1415 takes the pawn at (1,5) with the pawn (1,4)
3121
[[0. 1. 1. 2. 1.]
 [1. 1. 2. 1. 2.]
 [0. 2. 1. 2. 1.]
 [2. 1. 0. 1. 2.]
 [1. 2. 2. 2. 1.]]
Press Enter to see the computer's move.
[[0. 1. 1. 2. 1.]
 [1. 1. 2. 1. 2.]
 [0. 2. 1. 0. 2.]
 [2. 1. 0. 1. 2.]
 [1. 2. 2. 2. 1.]]
Enter your move, or type quit to end th